<a href="https://colab.research.google.com/github/GrigoryBartosh/dul_2021/blob/main/Homework/hw11/Homework11_selfsupervised2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!if [ -d dul_2021 ]; then rm -Rf dul_2021; fi
!git clone https://github.com/GrigoryBartosh/dul_2021
!pip install ./dul_2021

Cloning into 'dul_2021'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (117/117), done.
ceiving objects:   5% (19/361), 3.00 MiB | 1.43 MiB/sReceiving objects:   9% (33/361), 3.00 MiB | 1.43 MiB/sReceiving objects:  19% (69/361), 3.00 MiB | 1.43 MiB/sReceiving objects:  21% (76/361), 4.54 MiB | 1.74 MiB/sReceiving objects:  25% (91/361), 4.54 MiB | 1.74 MiB/sReceiving objects:  27% (101/361), 6.07 MiB | 1.95 MiB/sReceiving objects:  27% (101/361), 10.92 MiB | 2.28 MiB/sReceiving objects:  27% (101/361), 14.60 MiB | 2.16 MiB/sReceiving objects:  27% (101/361), 17.68 MiB | 1.67 MiB/sReceiving objects:  27% (101/361), 20.01 MiB | 1.32 MiB/sReceiving objects:  27% (101/361), 21.55 MiB | 1.14 MiB/sReceiving objects:  27% (101/361), 24.02 MiB | 1.07 MiB/sReceiving objects:  27% (101/361), 26.84 MiB | 1.20 MiB/sReceiving objects:  28% (102/361), 28.82 MiB | 1.30 MiB/sReceiving objects:  34% (124/361), 30.89 MiB | 1.

In [24]:
import os
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib
from tqdm.auto import tqdm

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils import data

import torchvision
from torchvision.datasets import STL10, CIFAR10
from torchvision import transforms

from sklearn.metrics import accuracy_score
from scipy.stats import mannwhitneyu
import numpy as np

from dul_2021.utils.hw11_utils import *

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Question 1. BYOL

Here we will implement [BYOL](https://arxiv.org/abs/2006.07733).

* You can combine view, representation, and projection into one network. You can use same architechure as in practice. 

* Use BatchNorm

* As predictor use few linear layers

* Dataset comes untransformed, so you need to apply transformations during training by yourself. Use same augmentations as in SimCLR

**Hyperparameters**

* τ = 0.99 (target update coefficient)
* lr = 1e-4
* num_epochs = 20
* latent dim = 128



**You will provide the following deliverables**


1. Over the course of training, record loss ber batch.
2. A function that encodes a batch of images with your trained model. The function recieves a batch torch tensors on cpu and should return transformed 2d tensor (batch size x laten dim). It will be used to test representation on classification task.

In [57]:
class ConvNet(nn.Module):
    def __init__(self, in_dim=1, out_dim=128, hid_dim_full=128):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(in_dim, 16, 5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 16, 3, padding=1, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.Conv2d(16, 32, 5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, 32, 3, padding=1, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, 32, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, 4, 1),
            nn.BatchNorm2d(4),
            nn.ReLU(True),
            nn.Flatten(),
            nn.Linear(4 * 8 * 8, hid_dim_full),
            nn.BatchNorm1d(hid_dim_full),
            nn.ReLU(True),
            nn.Linear(hid_dim_full, out_dim),
        )

    def forward(self, x):
        return self.seq(x)

In [58]:
def _identity(x): return x

class SSDatasetWrapper(data.Dataset):
    def __init__(self, base, transform=_identity):
        self._base = base
        self._transform = transform

    def __getitem__(self, idx):
        orig_data, _ = self._base[idx]
        return self._transform(orig_data), self._transform(orig_data)

    def __len__(self):
        return len(self._base)

In [59]:
class BYOL(nn.Module):
    def __init__(self, id_=1, hd=128, od=32, gamma=0.99):
        super().__init__()
        self._gamma = gamma

        self.student = ConvNet(id_, od, hd)

        self.teacher = ConvNet(id_, od, hd)
        for p in self.teacher.parameters():
            p.requires_grad_(False)

        self.predictor = nn.Sequential(
            nn.Linear(od, 2 * od),
            nn.BatchNorm1d(2 * od),
            nn.ReLU(True),
            nn.Linear(2 * od, od),
        )

    @property
    def device(self):
        return next(self.parameters()).device

    def update_teacher(self):
        for tp, sp in zip(self.teacher.parameters(), self.student.parameters()):
            tp.data = tp * self._gamma + sp * (1 - self._gamma)

    def _one_sided_loss(self, x, y):
        pr = self.predictor(self.student(x))
        gt = self.teacher(y)
        return 2 * (1 - (F.normalize(pr, dim=1) * F.normalize(gt, dim=1)).sum(-1))

    def _loss(self, x, y):
        return (self._one_sided_loss(x, y) + self._one_sided_loss(y, x)).mean()

    def fit(self, train_dl, n_epochs=10, lr=1e-3, wd=0.):
        losses = []
        opt = optim.AdamW(self.parameters(), lr=lr, weight_decay=wd)
        with tqdm(total=n_epochs * len(train_dl)) as bar:
            for _ in range(n_epochs):
                for batch in train_dl:
                    x, y = batch
                    x, y = x.to(self.device), y.to(self.device)

                    loss = self._loss(x, y)
                    loss.backward()
                    opt.step()
                    opt.zero_grad()

                    losses.append(loss.item())

                    bar.update(1)
                    bar.set_postfix(dict(loss=round(losses[-1], 5)))

        return np.array(losses)

    def forward(self, x):
        return self.student(x)

In [43]:
def q1(train_data):
    """
    train_data: An (n_train, 1, 28, 28) torchvision dataset of MNIST images with values from -1 to 1

    Returns
    - a (# of training iterations, ) numpy array  losses on each iteration
    - a function that transforms batch of images into their latent representation
    """
    contrast_transforms = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.RandomResizedCrop(size=32),
        ]
    )
    dataset = SSDatasetWrapper(train_data, transform=contrast_transforms)
    train_dl = data.DataLoader(dataset, batch_size=256, drop_last=True, shuffle=True)

    byol = BYOL().to(DEVICE)

    losses = byol.fit(train_dl)
    byol.eval()

    @torch.no_grad()
    def _encode(x):
        return byol(x).cpu()

    return losses, _encode

In [44]:
# change flag to False if you want only to test your losses w/o accuracy (it takes around 4-5 minutes)
q1_results(q1, True)

  0%|          | 0/2340 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Question 2. Barlow Twins

Here we will implement [barlow twins](https://arxiv.org/abs/2103.03230).

* You can use same architechure as in practice. 

* Dataset comes untransformed, so you need to apply transformations during training by yourself. Use same augmentations as in SimCLR

**Hyperparameters**

* λ = 0.01 
* lr = 5e-4
* num_epochs = 20
* latent dim = 128



**You will provide the following deliverables**


1. Over the course of training, record loss ber batch.
2. A function that encodes a batch of images with your trained model. The function recieves a batch torch tensors on cpu and should return transformed 2d tensor (batch size x laten dim). It will be used to test representation on classification task.

In [60]:
class BarlowTwins(nn.Module):
    def __init__(self, id_=1, hd=128, od=32, lambda_=0.9):
        super().__init__()
        self._lambda = lambda_

        self.encoder = ConvNet(id_, od, hd)

        self.projector = nn.Sequential(
            nn.Linear(od, 2 * od),
            nn.BatchNorm1d(2 * od),
            nn.ReLU(True),
            nn.Linear(2 * od, od),
        )

    @property
    def device(self):
        return next(self.parameters()).device

    def _loss(self, x, y):
        x_norm = F.normalize(self.projector(self.encoder(x)), dim=1)
        y_norm = F.normalize(self.projector(self.encoder(y)), dim=1)

        cc = x_norm.T @ y_norm
        N = cc.size(0)
        loss = (cc - torch.eye(N)).pow(2).mul_(self._lambda)
        loss[range(N), range(N)] /= self._lambda
        return loss.sum()

    def fit(self, train_dl, n_epochs=10, lr=1e-3, wd=0.):
        losses = []
        opt = optim.AdamW(self.parameters(), lr=lr, weight_decay=wd)
        with tqdm(total=n_epochs * len(train_dl)) as bar:
            for _ in range(n_epochs):
                for batch in train_dl:
                    x, y = batch
                    x, y = x.to(self.device), y.to(self.device)

                    loss = self._loss(x, y)
                    loss.backward()
                    opt.step()
                    opt.zero_grad()

                    losses.append(loss.item())

                    bar.update(1)
                    bar.set_postfix(dict(loss=round(losses[-1], 5)))

        return np.array(losses)

    def forward(self, x):
        return self.encoder(x)


In [63]:
def q2(train_data, _):
    """
    train_data: An (n_train, 1, 32, 32) torchvision dataset of CIFAR10 images with values from -1 to 1

    Returns
    - a (# of training iterations, ) numpy array  losses on each iteration
    - a function that transforms batch of images into their latent representation
    """

    contrast_transforms = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.RandomResizedCrop(size=32),
        ]
    )
    dataset = SSDatasetWrapper(train_data, transform=contrast_transforms)
    train_dl = data.DataLoader(dataset, batch_size=256, drop_last=True, shuffle=True)

    barlow_twins = BarlowTwins(id_=3).to(DEVICE)

    losses = barlow_twins.fit(train_dl)
    barlow_twins.eval()

    @torch.no_grad()
    def _encode(x):
        return barlow_twins(x).cpu()

    return losses, _encode

In [64]:
# change flag to False if you want only to test your losses w/o accuracy (it takes around 4-5 minutes)
q2_results(q2, True)

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/1950 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Bonus. SwAV

Here we will implement [SwAV](https://arxiv.org/abs/2006.09882v5)

**You will provide the following deliverables**


1. Over the course of training, record loss ber batch.
2. A function that encodes a batch of images with your trained model. The function recieves a batch torch tensors on cpu and should return transformed 2d tensor (batch size x laten dim). It will be used to test representation on classification task.

In [50]:
def b(train_data):
    """
    train_data: An (n_train, 1, 32, 32) torchvision dataset of CIFAR10 images with values from -1 to 1

    Returns
    - a (# of training iterations, ) numpy array  losses on each iteration
    - a function that transforms batch of images into their latent representation
    """

In [51]:
q2_results(b, True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./


KeyboardInterrupt: 